<a href="https://colab.research.google.com/github/Tamira819/chat_ESCOLA/blob/main/chatEscola.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# O chatEscola é um rescurso de IA para auxiliar alunos e responsáveis na realização de tarefas escolares e reforço de aprendizado.

### Muitas famílias passam por dificuldades na hora de fazer o para casa, trabalhos escolares e de estudar antes de provas e avalições. Uma atividade com erro, um enunciado ambíguo, um conteúdo que o aluno não entendeu bem e o responsável já não se lembra mais...

### Antigamente, as famílias podiam consultar a "Barsa", ou consultar outros livros que estivessem disponíveis.

### Hoje em dia, a IA pode facilicitar a busca de informações, trazendo conteúdo de maneira didática, com qualidade e confiança, numa velocidade quase instantânea.




Começando pelo começo, vamos fazer as instalações

In [60]:
!pip install -q -u google-generative-ai


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -u


In [61]:
import google.generativeai as genai
import numpy as np
import pandas as pd
from google.colab import userdata

api_key = userdata.get("google_api_key")
genai.configure(api_key=api_key)

Agora vamos estabelecer configurações de máxima segurança, pois este deve ser um ambiente educativo, próprio para menores de idade.

In [62]:
generation_config = {
    'candidate_count': 1,
    'temperature': 0,
    }

safety_settings = {
    'HARASSMENT': "BLOCK_MOST",
    'HATE': "BLOCK_MOST",
    'DANGEROUS': "BLOCK_MOST",
    'SEXUAL': "BLOCK_MOST",
}

Aqui podemos começar a preparar a nossa base de dados para consulta, que podem ser documentos e livros didáticos digitalizados, artigos e livros disponíveis online.
Infelizmente o tempo de elaboração deste projeto foi muito curto, então vou apenas simular os resultados.

Dados retirados do site https://brasilescola.uol.com.br/

In [63]:
document1 = {
    "Título": "Área do Triângulo",
    "Conteúdo": "A fórmula que pode ser usada para calcular a área do triângulo é a seguinte: area = (b * h) / 2. Nessa fórmula, b é a medida da base do triângulo e h é a medida de sua altura."
}

document2 = {
    "Título": "O que é antibiótico?",
    "Conteúdo": "Os antibióticos são medicamentos de origem natural ou sintética que inibem o crescimento ou causam a morte de bactérias."
}

document3 = {
    "Título": "O que é Concordância Nominal?",
    "Conteúdo": "Todos os elementos que compõem a oração precisam estar em harmonia, ou seja, em concordância. Quando esta estiver relacionada ao sintagma nominal (parte da oração cujo núcleo é o substantivo), há a Concordância Nominal. A regra geral de Concordância Nominal diz que todos os determinantes (adjetivo, numeral, pronome adjetivo e artigo) devem harmonizar-se quanto ao gênero e ao número do substantivo."
}

Em seguida, vamos começar o processar de embed.

In [64]:
documents = [document1, document2, document3]

df = pd.DataFrame(documents)
df

,Título,Conteúdo
0,Área do Triângulo,A fórmula que pode ser usada para calcular a á...
1,O que é antibiótico?,Os antibióticos são medicamentos de origem nat...
2,O que é Concordância Nominal?,Todos os elementos que compõem a oração precis...


In [65]:
model = 'models/embedding-001'


In [66]:
def embed_fn(title, text):
  return genai.embed_content(model = model, content = text, title = title, task_type= "RETRIEVAL_DOCUMENT")["embedding"]

In [67]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row["Título"], row["Conteúdo"]), axis=1)
df

,Título,Conteúdo,Embeddings
0,Área do Triângulo,A fórmula que pode ser usada para calcular a á...,"[0.02472404, -0.074365795, 0.011906829, 0.0305..."
1,O que é antibiótico?,Os antibióticos são medicamentos de origem nat...,"[0.0034523783, -0.08416602, -0.025197366, 0.04..."
2,O que é Concordância Nominal?,Todos os elementos que compõem a oração precis...,"[0.01013491, -0.04660659, -0.045423217, 0.0426..."


In [68]:
def gerar_consulta(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model = model, content = consulta, task_type= "RETRIEVAL_QUERY")
  produtos_escalares = np.dot(np.stack(df["Embeddings"]), embedding_da_consulta["embedding"])
  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]["Conteúdo"]

In [69]:
generation_config = {
    'candidate_count': 1,
    'temperature': 0,
    }
safety_settings = {
    'HARASSMENT': "BLOCK_MOST",
    'HATE': "BLOCK_MOST",
    'SEXUAL': "BLOCK_MOST",
    'DANGEROUS': "BLOCK_MOST"
}

Habilitando o chat interativo.

Aqui eu tive muita dificuldade em habilidade a função de histórico do chat, como explicado na aula 4, porém o while aqui no notebook mantém o histórico até o final da iteração.
Consequentemente também não consegui utilizar o trecho para modificar o markdown e melhorar o visual do chat.

In [88]:
chat = model.start_chat(history=[])

AttributeError: 'str' object has no attribute 'start_chat'

In [86]:
from os import replace
import textwrap
from IPython.display import Markdown, display

def to_markdown(text):
  text = text.replace('°', ' *')
  return Markdown(textwrap.indent(text, '>', predicate = lambda _: True))

for message in chat.history:
  display(to_markdown(f'**{message.role}**: {message.parts[0].text}'))
  print('-----------')


In [89]:
consulta = input('o que você quer saber hoje? (digite fim para sair =) ')

while consulta != 'fim':
  #response = chat.send_message(prompt)
  trecho = gerar_consulta(consulta, df, model)
  prompt = f"Reescreva esse texto de uma forma mais didática, sem adicionar informações {trecho}"

  model_2 = genai.GenerativeModel("gemini-1.0-pro",
                              generation_config=generation_config,)
  response = model_2.generate_content(prompt)
  print(response.text)
  consulta = input('o que você quer saber hoje? (digite fim para sair =) ')



o que você quer saber hoje? (digite fim para sair =) antibiotico
**O que são antibióticos?**

Os antibióticos são remédios que ajudam a combater as bactérias. Eles podem ser feitos de plantas ou criados em laboratório.

**Como os antibióticos funcionam?**

Os antibióticos agem de duas maneiras:

* **Parando o crescimento das bactérias:** Eles impedem que as bactérias se multipliquem e se espalhem.
* **Matando as bactérias:** Eles destroem as bactérias, impedindo-as de causar infecções.
o que você quer saber hoje? (digite fim para sair =) fim
